In [1]:
import asyncio
import asyncpg

import pandas as pd
import numpy as np

In [2]:
import matplotlib.pyplot as plt

In [3]:
data = pd.read_csv('data.csv')

###  В данных есть пустые заказы, но их мало

In [4]:
print('количество пустых заказов: ', (data['total_quantity'] == 0).sum())

количество пустых заказов:  29


In [5]:
data[data['total_quantity'] == 0].groupby(['retailer_id', 'store_id']).count()

Unnamed: 0  id  user_id  total_cost  cost  item_total  \
retailer_id store_id                                                          
1.0         1                  1   1        1           1     1           1   
            2                  1   1        1           1     1           1   
            3                  6   6        6           6     6           6   
            8                  6   6        6           6     6           6   
            10                 3   3        3           3     3           3   
            11                 1   1        1           1     1           1   
            12                 3   3        3           3     3           3   
            14                 1   1        1           1     1           1   
            18                 1   1        1           1     1           1   
            21                 2   2        2           2     2           2   
            98                 4   4        4           4     4           4   

                      promo_total  total_weight  total_quantity  state  ...  \
retailer_id store_id                                                    ...   
1.0         1                   1             1               1      1  ...   
            2                   1             1               1      1  ...   
            3                   6             6               6      6  ...   
            8                   6             6               6      6  ...   
            10                  3             3               3      3  ...   
            11                  1             1               1      1  ...   
            12                  3             3               3      3  ...   
            14                  1             1               1      1  ...   
            18                  1             1               1      1  ...   
            21                  2             2               2      2  ...   
            98                  4             4               4      4  ...   

                      item_total_class  in_time  delta_t  promo_count  \
retailer_id store_id                                                    
1.0         1                        1        1        1            1   
            2                        1        1        1            1   
            3                        6        6        6            6   
            8                        6        6        6            6   
            10                       3        3        3            3   
            11                       1        1        1            1   
            12                       3        3        3            3   
            14                       1        1        1            1   
            18                       1        1        1            1   
            21                       2        2        2            2   
            98                       4        4        4            4   

                      order_count  old_client  item_total_avg  user_class  \
retailer_id store_id                                                        
1.0         1                   1           1               1           1   
            2                   1           1               1           1   
            3                   6           6               6           6   
            8                   6           6               6           6   
            10                  3           3               3           3   
            11                  1           1               1           1   
            12                  3           3               3           3   
            14                  1           1               1           1   
            18                  1           1               1           1   
            21                  2           2               2           2   
            98                  4           4               4           4   

              

In [6]:
print('распределение пустых заказов по магазинам')
data[data['state'] == 'canceled']['store_id']

распределение пустых заказов по магазинам


627       10
31719     14
47726      1
68400     58
168660     8
169944    18
183126    14
211247    18
216071    14
224511    12
224880    12
250659    21
252205    14
Name: store_id, dtype: int64

In [7]:
print('распределение пустых заказов по ритейлерам')
data[data['state'] == 'canceled']['retailer_id']

распределение пустых заказов по ритейлерам


627       1.0
31719     1.0
47726     1.0
68400     NaN
168660    1.0
169944    1.0
183126    1.0
211247    1.0
216071    1.0
224511    1.0
224880    1.0
250659    1.0
252205    1.0
Name: retailer_id, dtype: float64

они все у одного ритейлера, но по нему самая большая статистика, так что неудивительно

In [8]:
data = data[data['total_quantity'] != 0]
data['replace_freq'] = data['replaced_count'] / data['total_quantity']
data['cancel_freq'] = data['canceled_count'] / data['total_quantity']

### Посчитаем квантили по выручке, чееку, делям отмены и замены, числу товаров

In [9]:
def q10(x):
    return x.quantile(0.1)

def q50(x):
    return x.quantile(0.5)

def q75(x):
    return x.quantile(0.75)

def q95(x):
    return x.quantile(0.95)

In [10]:
data.groupby(['retailer_id'])\
['total_cost', 'item_total', 'cancel_freq', 'replace_freq', 'total_quantity']\
.agg(['count', q10, q50, q95]).T

retailer_id                    1.0          8.0          15.0         16.0
total_cost     count  323398.000000  1436.000000  7574.000000   192.000000
               q10         0.000000     0.000000     0.000000     0.000000
               q50       199.000000   290.000000   199.000000   158.000000
               q95       447.000000   398.000000   398.000000   299.000000
item_total     count  323398.000000  1436.000000  7574.000000   192.000000
               q10      1684.000000  1076.500000  1365.600000  1059.800000
               q50      3912.000000  2817.000000  2986.500000  2843.000000
               q95     11483.150000  7025.500000  7091.700000  6023.500000
cancel_freq    count  323398.000000  1436.000000  7574.000000   192.000000
               q10         0.000000     0.000000     0.000000     0.000000
               q50         0.020000     0.075000     0.046512     0.000000
               q95         0.200000     0.400000     0.303030     0.222500
replace_freq   count  323398.000000  1436.000000  7574.000000   192.000000
               q10         0.000000     0.000000     0.000000     0.000000
               q50         0.000000     0.028992     0.030303     0.000000
               q95         0.133333     0.250000     0.181818     0.234722
total_quantity count  323398.000000  1436.000000  7574.000000   192.000000
               q10         9.000000     9.000000    12.000000     5.000000
               q50        31.000000    22.000000    28.000000    14.000000
               q95       112.000000    63.250000    75.000000    36.900000

### у ритейлеров 8 и 15 достаточно много замен и отмен по сравнению с 1 (у ритейлера 16 мало статистики)

In [11]:
store_stats = data.groupby(['store_id'])['total_cost', 'item_total', 'cancel_freq', 'replace_freq'].agg(['count', q50, q95])
store_stats.T

store_id                    1             2             3            6    \
total_cost   count  26758.00000  34537.000000  19411.000000   541.000000   
             q50      249.00000    249.000000    199.000000   278.000000   
             q95      459.00000    479.000000    404.000000   388.000000   
item_total   count  26758.00000  34537.000000  19411.000000   541.000000   
             q50     4033.50000   4058.000000   3598.000000  2837.000000   
             q95    11745.45000  11977.800000   9689.500000  8156.000000   
cancel_freq  count  26758.00000  34537.000000  19411.000000   541.000000   
             q50        0.02000      0.016129      0.030303     0.062112   
             q95        0.18750      0.166667      0.250000     0.309524   
replace_freq count  26758.00000  34537.000000  19411.000000   541.000000   
             q50        0.00000      0.000000      0.000000     0.046512   
             q95        0.12766      0.117647      0.125992     0.181818   

store_id                     8             9             10            11   \
total_cost   count  28271.000000  13387.000000  22911.000000  21430.000000   
             q50      249.000000    199.000000    199.000000    224.000000   
             q95      448.000000    409.000000    414.000000    448.000000   
item_total   count  28271.000000  13387.000000  22911.000000  21430.000000   
             q50     3865.000000   3887.000000   4017.000000   4128.000000   
             q95    10649.000000  10468.000000  11159.000000  11208.000000   
cancel_freq  count  28271.000000  13387.000000  22911.000000  21430.000000   
             q50        0.023256      0.023256      0.022727      0.027523   
             q95        0.222222      0.205882      0.208333      0.206897   
replace_freq count  28271.000000  13387.000000  22911.000000  21430.000000   
             q50        0.000000      0.000000      0.000000      0.000000   
             q95        0.133333      0.115385      0.142857      0.130435   

store_id                     12            14   ...          95           98   \
total_cost   count  34331.000000  34740.000000  ...  1994.000000  5013.000000   
             q50      249.000000    249.000000  ...    99.000000    99.000000   
             q95      489.000000    469.000000  ...   319.000000   158.000000   
item_total   count  34331.000000  34740.000000  ...  1994.000000  5013.000000   
             q50     4276.000000   4091.000000  ...  3736.000000  3144.000000   
             q95    14197.000000  11567.000000  ...  9946.500000  7694.600000   
cancel_freq  count  34331.000000  34740.000000  ...  1994.000000  5013.000000   
             q50        0.000000      0.021739  ...     0.009662     0.035714   
             q95        0.173913      0.210526  ...     0.182997     0.307692   
replace_freq count  34331.000000  34740.000000  ...  1994.000000  5013.000000   
             q50        0.000000      0.000000  ...     0.000000     0.000000   
             q95        0.132843      0.125000  ...     0.106649     0.130435   

store_id                    99           100          105          108  \
total_cost   count    41.000000    58.000000  6662.000000  3936.000000   
             q50       0.000000     0.000000    99.000000    99.000000   
             q95     199.000000   199.000000   158.000000   158.000000   
item_total   count    41.000000    58.000000  6662.000000  3936.000000   
             q50    2069.000000  2163.500000  3152.000000  3088.500000   
             q95    4967.000000  4668.150000  7757.300000  7809.000000   
cancel_freq  count    41.000000    58.000000  6662.000000  3936.000000   
             q50       0.071429     0.052632     0.038462     0.000000   
             q95       0.363636     0.241855     0.250000     0.187500   
replace_freq count    41.000000    58.000000  6662.000000  3936.000000   
             q50       0.136364     0.109610     0.000000     0.017241   
             q95       0.375000     0.281818     0.111

### Магазинов много, поэтому можно посчитать среднее и дисперсию и посмотреть отклонение больше одного стандартного

* replace_freq

In [12]:
avg = store_stats.mean()
std = store_stats.std()
normalized_store_stats = (store_stats - avg) / std
print(avg, std)
store_stats[normalized_store_stats['replace_freq']['q50'].abs() > 1]

total_cost    count    10127.757576
              q50        168.484848
              q95        329.140909
item_total    count    10127.757576
              q50       3326.939394
              q95       8766.627273
cancel_freq   count    10127.757576
              q50          0.028053
              q95          0.231953
replace_freq  count    10127.757576
              q50          0.020162
              q95          0.169612
dtype: float64 total_cost    count    12177.019496
              q50         83.696670
              q95        122.791413
item_total    count    12177.019496
              q50        636.276580
              q95       2550.155990
cancel_freq   count    12177.019496
              q50          0.022284
              q95          0.071296
replace_freq  count    12177.019496
              q50          0.032472
              q95          0.068534
dtype: float64


total_cost                item_total                  cancel_freq  \
              count    q50     q95      count     q50      q95       count   
store_id                                                                     
56              558  199.0  304.55        558  2456.0  6199.45         558   
99               41    0.0  199.00         41  2069.0  4967.00          41   
100              58    0.0  199.00         58  2163.5  4668.15          58   

                             replace_freq                      
               q50       q95        count       q50       q95  
store_id                                                       
56        0.071946  0.386923          558  0.058824  0.277778  
99        0.071429  0.363636           41  0.136364  0.375000  
100       0.052632  0.241855           58  0.109610  0.281818

### Магазин 56 часто заменяет товары

* cancel_freq

In [13]:
avg = store_stats.mean()
std = store_stats.std()
normalized_store_stats = (store_stats - avg) / std
print(avg, std)
df = store_stats[normalized_store_stats['cancel_freq']['q95'].abs() > 1]
df[df['cancel_freq']['count'] > 500]

total_cost    count    10127.757576
              q50        168.484848
              q95        329.140909
item_total    count    10127.757576
              q50       3326.939394
              q95       8766.627273
cancel_freq   count    10127.757576
              q50          0.028053
              q95          0.231953
replace_freq  count    10127.757576
              q50          0.020162
              q95          0.169612
dtype: float64 total_cost    count    12177.019496
              q50         83.696670
              q95        122.791413
item_total    count    12177.019496
              q50        636.276580
              q95       2550.155990
cancel_freq   count    12177.019496
              q50          0.022284
              q95          0.071296
replace_freq  count    12177.019496
              q50          0.032472
              q95          0.068534
dtype: float64


total_cost                item_total                  cancel_freq  \
              count    q50     q95      count     q50      q95       count   
store_id                                                                     
6               541  278.0  388.00        541  2837.0  8156.00         541   
18              542  299.0  499.00        542  3221.0  7644.85         542   
56              558  199.0  304.55        558  2456.0  6199.45         558   
57             1337  299.0  398.00       1337  2856.0  7376.20        1337   
58              517    0.0  382.00        517  2635.0  6851.60         517   
70             6574  249.0  429.00       6574  3064.0  7251.70        6574   
98             5013   99.0  158.00       5013  3144.0  7694.60        5013   
118            2113   99.0  178.40       2113  3349.0  8956.80        2113   

                             replace_freq                      
               q50       q95        count       q50       q95  
store_id                                                       
6         0.062112  0.309524          541  0.046512  0.181818  
18        0.027778  0.095022          542  0.027778  0.083114  
56        0.071946  0.386923          558  0.058824  0.277778  
57        0.076923  0.400000         1337  0.019608  0.222222  
58        0.062500  0.308654          517  0.045455  0.222222  
70        0.047619  0.312500         6574  0.027027  0.176471  
98        0.035714  0.307692         5013  0.000000  0.130435  
118       0.002370  0.142857         2113  0.000000  0.100000

### 118, 12, 108 почти не отменяют
### 6, 56, 57, 58 много отменяют

## Посмотрим на delta_t (величина, которая показывает отклонение времени доставки от окна доставки)

In [14]:
store_stats = data.groupby(['store_id'])['total_cost', 'item_total', 'cancel_freq', 'replace_freq', 'delta_t'].agg(['count', q50, q75])
avg = store_stats.mean()
std = store_stats.std()
normalized_store_stats = (store_stats - avg) / std
store_stats.T

store_id                     1             2             3            6    \
total_cost   count  26758.000000  34537.000000  19411.000000   541.000000   
             q50      249.000000    249.000000    199.000000   278.000000   
             q75      299.000000    299.000000    299.000000   299.000000   
item_total   count  26758.000000  34537.000000  19411.000000   541.000000   
             q50     4033.500000   4058.000000   3598.000000  2837.000000   
             q75     6011.750000   6029.000000   5258.000000  4419.000000   
cancel_freq  count  26758.000000  34537.000000  19411.000000   541.000000   
             q50        0.020000      0.016129      0.030303     0.062112   
             q75        0.064516      0.057692      0.086957     0.125000   
replace_freq count  26758.000000  34537.000000  19411.000000   541.000000   
             q50        0.000000      0.000000      0.000000     0.046512   
             q75        0.042216      0.037037      0.038462     0.093750   
delta_t      count  26758.000000  34537.000000  19411.000000   541.000000   
             q50        0.000000      0.000000      0.000000     0.000000   
             q75        0.000000      0.000000      0.000000     0.000000   

store_id                     8             9             10            11   \
total_cost   count  28271.000000  13387.000000  22911.000000  21430.000000   
             q50      249.000000    199.000000    199.000000    224.000000   
             q75      299.000000    299.000000    299.000000    299.000000   
item_total   count  28271.000000  13387.000000  22911.000000  21430.000000   
             q50     3865.000000   3887.000000   4017.000000   4128.000000   
             q75     5720.000000   5664.500000   6042.000000   6066.750000   
cancel_freq  count  28271.000000  13387.000000  22911.000000  21430.000000   
             q50        0.023256      0.023256      0.022727      0.027523   
             q75        0.074074      0.069767      0.071429      0.075000   
replace_freq count  28271.000000  13387.000000  22911.000000  21430.000000   
             q50        0.000000      0.000000      0.000000      0.000000   
             q75        0.043478      0.037037      0.047619      0.046512   
delta_t      count  28271.000000  13387.000000  22911.000000  21430.000000   
             q50        0.000000      0.000000      0.000000      0.000000   
             q75        0.000000      0.000000      0.000000      0.000000   

store_id                     12            14   ...          95           98   \
total_cost   count  34331.000000  34740.000000  ...  1994.000000  5013.000000   
             q50      249.000000    249.000000  ...    99.000000    99.000000   
             q75      299.000000    299.000000  ...   299.000000   149.000000   
item_total   count  34331.000000  34740.000000  ...  1994.000000  5013.000000   
             q50     4276.000000   4091.000000  ...  3736.000000  3144.000000   
             q75     6574.500000   6110.500000  ...  5479.750000  4508.000000   
cancel_freq  count  34331.000000  34740.000000  ...  1994.000000  5013.000000   
             q50        0.000000      0.021739  ...     0.009662     0.035714   
             q75        0.052632      0.071429  ...     0.057143     0.100000   
replace_freq count  34331.000000  34740.000000  ...  1994.000000  5013.000000   
             q50        0.000000      0.000000  ...     0.000000     0.000000   
             q75        0.039216      0.040000  ...     0.037736     0.040000   
delta_t      count  34331.000000  34740.000000  ...  1994.000000  5013.000000   
             q50        0.000000      0.000000  ...     0.000000     0.000000   
             q75        0.000000      0.000000  ...     0.000000     0.197222   

store_id                    99           100          105          108  \
total_cost   count    41.000000    58.000000  6662.000000  3936.000000   
             q50       0.000000     0.000000    99.000000    99.000000   

In [15]:
print('средний 0.5 квантиль: ', avg['delta_t']['q50'], '\nотклонение 0.5 квантиля: ', std['delta_t']['q50'])
df = store_stats[normalized_store_stats['delta_t']['q50'].abs() > 1]
df[df['delta_t']['count'] > 500]

средний 0.5 квантиль:  -0.05869528619528619 
отклонение 0.5 квантиля:  0.3371787486053004


total_cost               item_total                 cancel_freq  \
              count    q50    q75      count     q50     q75       count   
store_id                                                                   
18              542  299.0  299.0        542  3221.0  4677.5         542   

                             replace_freq                     delta_t  \
               q50       q75        count       q50       q75   count   
store_id                                                                
18        0.027778  0.055556          542  0.027778  0.055556     542   

                              
               q50       q75  
store_id                      
18       -1.936944 -1.789792

### 18 магазин привозит на 2 часа раньше в половине случаев

In [16]:
print('средний 0.75 квантиль: ', avg['delta_t']['q75'], '\nотклонение 0.75 квантиля: ', std['delta_t']['q75'])
df = store_stats[normalized_store_stats['delta_t']['q75'].abs() > 1]
df[df['delta_t']['count'] > 500]

средний 0.75 квантиль:  0.013705808080808084 
отклонение 0.75 квантиля:  0.41007701020949144


total_cost               item_total                 cancel_freq  \
              count    q50    q75      count     q50     q75       count   
store_id                                                                   
18              542  299.0  299.0        542  3221.0  4677.5         542   
57             1337  299.0  299.0       1337  2856.0  4148.0        1337   

                             replace_freq                     delta_t  \
               q50       q75        count       q50       q75   count   
store_id                                                                
18        0.027778  0.055556          542  0.027778  0.055556     542   
57        0.076923  0.163636         1337  0.019608  0.092105    1337   

                              
               q50       q75  
store_id                      
18       -1.936944 -1.789792  
57        0.000000  1.417778

### 57 магазин привозит на 1.41 часа позже в 1/4 случаев

In [17]:
data.groupby(['retailer_id', 'in_time']).count()['id'] / data.groupby('retailer_id').count()['id']

retailer_id  in_time
1.0          earlier    0.095331
             in_time    0.745595
             later      0.159073
8.0          earlier    0.113510
             in_time    0.486072
             later      0.400418
15.0         earlier    0.136784
             in_time    0.776076
             later      0.087140
16.0         earlier    0.093750
             in_time    0.593750
             later      0.312500
Name: id, dtype: float64

### 8 ритейлер в 40% случаев опаздывает

In [18]:
in_time_stats = data.groupby(['in_time', 'store_id']).count()['id'] / data.groupby('store_id').count()[['id']].T
in_time_stats

in_time    earlier                                                            \
store_id       1       2         3         6         8         9         10    
id        0.057553  0.0745  0.111535  0.460259  0.079587  0.149249  0.086683   

in_time                                 ...     later                      \
store_id       11        12        14   ...       95        98        99    
id        0.072795  0.110804  0.093408  ...  0.168004  0.309595  0.097561   

in_time                                                                      
store_id       100       105       108       112       113       118    120  
id        0.034483  0.205944  0.241108  0.333333  0.115385  0.098438  0.375  

[1 rows x 99 columns]

In [19]:
in_time_stats = data.groupby(['in_time', 'store_id']).count()['id'] / data.groupby('store_id').count()[['id']].T

unusual_stores = dict()
for in_time in pd.unique(data['in_time']):
    avg = in_time_stats[in_time].T.mean()
    std = in_time_stats[in_time].T.std()
    idxs = (in_time_stats[in_time].T - avg) / std > 1
    unusual_stores[in_time] = list(idxs[idxs['id']].index)
unusual_stores

{'earlier': [6, 18, 100, 113],
 'later': [57, 58, 98, 112, 120],
 'in_time': [118]}

### Статистика по магазинам:
### 6, 18, 100, 113 часто привозят раньше
### 57, 58, 98, 112, 120 часто привозят позже
### 118 привозит вовремя

## Посмотрим на прибыль

In [20]:
data.groupby('retailer_id').agg(['mean', q10, q50, q75, q95])['total_cost']

,mean,q10,q50,q75,q95
retailer_id,,,,,
1.0,189.734721,0.0,199.0,299.0,447.0
8.0,183.738858,0.0,290.0,299.0,398.0
15.0,167.676129,0.0,199.0,299.0,398.0
16.0,161.468750,0.0,158.0,299.0,299.0


In [21]:
data.groupby('city').agg(['mean', q10, q50, q75, q95])['total_cost']

,mean,q10,q50,q75,q95
city,,,,,
0,201.207188,0.0,249.0,299.0,448.0
1,85.863355,0.0,99.0,149.0,158.0
2,85.978354,0.0,99.0,149.0,158.0
3,124.963794,0.0,149.0,199.0,227.0
4,233.428571,0.0,294.5,299.0,455.1
5,89.691905,0.0,99.0,149.0,178.0
6,97.879202,0.0,99.0,149.0,160.0
7,320.700000,268.3,290.0,299.0,501.3


### Прибыль по Москве выше, чем по другим городам

## Посмотрим, как показатели меняются со временем

In [22]:
df = data
df['is_in_time'] = (data['in_time'] == 'in_time')
data.groupby(['retailer_id', 'month']).agg(['count', 'mean'])[['total_cost', 'replace_freq', 'cancel_freq', 'is_in_time']]

total_cost             replace_freq           cancel_freq  \
                         count        mean        count      mean       count   
retailer_id month                                                               
1.0         2018-01       7959  189.913431         7959  0.016723        7959   
            2018-02       7299  183.153172         7299  0.016472        7299   
            2018-03       8136  211.961406         8136  0.019177        8136   
            2018-04      10088  213.740979        10088  0.028527       10088   
            2018-05      13071  185.166246        13071  0.037246       13071   
            2018-06      11675  224.008651        11675  0.039128       11675   
            2018-07      11666  238.239585        11666  0.040953       11666   
            2018-08      11707  221.657935        11707  0.059928       11707   
            2018-09      15905  195.752133        15905  0.044654       15905   
            2018-10      18513  196.738277        18513  0.042730       18513   
            2018-11      18584  197.361268        18584  0.028238       18584   
            2018-12      16181  191.384338        16181  0.028780       16181   
            2019-01      18250  194.434621        18250  0.029344       18250   
            2019-02      16722  195.610094        16722  0.031023       16722   
            2019-03      18038  200.191252        18038  0.030241       18038   
            2019-04      22048  187.473149        22048  0.030266       22048   
            2019-05      25571  187.481697        25571  0.026407       25571   
            2019-06      30210  171.950215        30210  0.024574       30210   
            2019-07      37786  149.169745        37786  0.021172       37786   
            2019-08       3989  128.240035         3989  0.029392        3989   
8.0         2018-01         76  149.723684           76  0.077788          76   
            2018-02         73  140.369863           73  0.062478          73   
            2018-03        128  190.039062          128  0.045575         128   
            2018-04        125  197.976000          125  0.056599         125   
            2018-05        166  134.740964          166  0.051156         166   
            2018-06        235  212.961702          235  0.029239         235   
            2018-07        121  210.206612          121  0.034977         121   
            2018-08         83  191.939759           83  0.103457          83   
            2018-09         94  175.893617           94  0.082641          94   
            2018-10         93  218.032258           93  0.071565          93   
            2018-11         61  235.114754           61  0.101950          61   
            2018-12        148  147.743243          148  0.123252         148   
            2019-01         33  192.515152           33  0.080614          33   
15.0        2018-05         37  244.756757           37  0.081054          37   
            2018-06        526  227.342205          526  0.057812         526   
            2018-07        569  219.942004          569  0.050793         569   
            2018-08        635  216.618898          635  0.062022         635   
            2018-09        660  183.313636          660  0.052452         660   
            2018-10        759  189.096179          759  0.045098         759   
            2018-11        723  168.381743          723  0.051260         723   
            2018-12        488  183.327869          488  0.049786         488   
            2019-01        403  229.369727          403  0.042873         403   
            2019-02        309  232.611650          309  0.052024         309   
            2019-03        301  216.800664          301  0.052464         301   
            2019-04        370  208.037838          370  0.059668         370   
            2019-05        386  200.448187          386  0.058863         386   
            2019-06        560   33.944643         

### не сильно меняются

## Посмотрим, как клиенты используют промокоды

### Процент клиентов, использующих промокоды от времени

In [23]:
data['is_promo'] = (data['cost'] != data['total_cost'])

In [24]:
data.groupby('month').agg('mean')['is_promo']

month
2018-01    0.000489
2018-02    0.001060
2018-03    0.001400
2018-04    0.002835
2018-05    0.003416
2018-06    0.007314
2018-07    0.039738
2018-08    0.047485
2018-09    0.050003
2018-10    0.135657
2018-11    0.401900
2018-12    0.364869
2019-01    0.273948
2019-02    0.234925
2019-03    0.217242
2019-04    0.453382
2019-05    0.437899
2019-06    0.534168
2019-07    0.690854
2019-08    0.714564
Name: is_promo, dtype: float64

### Как часто пользуются промокодами новые и постоянные клиенты?

In [25]:
first_order = data[['created_at', 'user_id']].groupby('user_id').min().reset_index()

In [26]:
data = pd.merge(data, first_order, how='left', on='user_id', suffixes=('', '_first'))

In [27]:
data['is_first_order'] = (data['created_at'] - data['created_at_first']).abs() < 0.01

### Какая доля постоянных клиентов пользовалась промокодом при первом заказе?

In [28]:
data[(data['month'] < '2019-05') & data['old_client'] & data['is_first_order']]['is_promo'].mean()

0.3333048164941398

###  Какая доля клиентов, которые больше не заказывали, пользовалась промоколом при своем единственном заказе?

In [29]:
data[(data['month'] < '2019-05') & ~data['old_client'] & data['is_first_order']]['is_promo'].mean()

0.4302516829865361

### Процент заказов, сделанных непостоянными клиентами

In [30]:
(data.shape[0] - data['old_client'].sum()) / data.shape[0]

0.2261142494674103

## Посмотрим на распределение времени доставки и размера заказа по дням недели

### дни недели заказа:

In [31]:
data.groupby('day_of_week').agg(['mean', 'count'])[['total_quantity', 'total_cost', 'is_in_time']]

total_quantity         total_cost        is_in_time       
                      mean  count        mean  count       mean  count
day_of_week                                                           
0                46.590113  46747  195.306775  46747   0.751877  46747
1                46.718581  46532  185.775556  46532   0.738288  46532
2                44.229718  45791  189.442074  45791   0.739163  45791
3                45.240063  49941  190.144007  49941   0.740994  49941
4                45.219906  51563  190.818901  51563   0.740415  51563
5                40.766651  48991  185.160608  48991   0.749730  48991
6                40.442745  44651  186.734564  44651   0.749546  44651

### дни недели доставки:

In [32]:
data.groupby('day_of_week_shipped').agg(['mean', 'count'])[['total_quantity', 'total_cost', 'is_in_time']]

total_quantity         total_cost        is_in_time       
                              mean  count        mean  count       mean  count
day_of_week_shipped                                                           
0                        46.680609  46861  194.772511  46861   0.754849  46861
1                        45.328850  46617  191.257993  46617   0.747538  46617
2                        46.379258  45800  188.953577  45800   0.741550  45800
3                        44.354244  45813  186.923536  45813   0.738524  45813
4                        45.292255  49361  189.717997  49361   0.735621  49361
5                        41.631864  50101  187.678048  50101   0.742520  50101
6                        40.129694  49663  184.549523  49663   0.749069  49663

### По выходным заказов больше, но они меньше. Но на то, доставляются ли они вовремя, день недели не влияет

## Рассмотрим заказы клиентов различных классов

In [33]:
data.groupby(['user_class', 'day_of_week_shipped']).agg(['mean', 'count'])[['is_in_time']]

is_in_time       
                                     mean  count
user_class day_of_week_shipped                  
high       0                     0.749773   9923
           1                     0.745360   9967
           2                     0.728921   9820
           3                     0.729045   9592
           4                     0.721332  10299
           5                     0.742618   8738
           6                     0.745886   8508
low        0                     0.748485   9073
           1                     0.737287   9341
           2                     0.729876   9603
           3                     0.730331   9393
           4                     0.729205   9786
           5                     0.725069   9737
           6                     0.734613   9895
middle     0                     0.758730  27865
           1                     0.751840  27309
           2                     0.750502  26377
           3                     0.744782  26828
           4                     0.742793  29276
           5                     0.747866  31626
           6                     0.754511  31260

### Клиенты, делающие дорогие заказы, на выходных делают их меньше
### Клиенты, делающие средние по величине заказы, на выходных делают их больше
### Клиенты, делающие дешевые заказы, примерно стабильно ведут себя всю неделю

In [34]:
data.groupby('user_class').mean()['is_promo']

user_class
high      0.262390
low       0.402331
middle    0.273999
Name: is_promo, dtype: float64

### Клиенты, делающие средние заказы, чаще пользуются промокодами

In [35]:
data.groupby(['user_class', 'old_client']).agg(['count', 'mean'])[['is_promo']]

is_promo          
                         count      mean
user_class old_client                   
high       False         10152  0.666470
           True          56695  0.190034
low        False         33615  0.563141
           True          33213  0.239575
middle     False         31804  0.612502
           True         168737  0.210197

### Среди клиентов, делающих средние и дорогие заказы, больше постоянных

# Конечно, можно было посмотреть на большее количество распределений и обосновать все более строго.